**Visualize all tracks in file '2017-05-17_00h_tracks.csv' :**

In [9]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import csv

num_TOs = 1500

# Loading original tracks data:
cols = ['ID','Time','x', 'y','Speed','Type','Estimated']
data = pd.read_csv('2017-05-12_00h_tracks.csv',index_col=False, header=None,sep=';',names=cols )
data = data.drop(data.index[0])   # drops the first row
df = data[(data['Type'] == 2)]  # only selecting TO of type 2 (car)


# creating new data frame for plotting trajectories: 
df = df[['ID', 'Time','x', 'y']]

print(df.head())

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_17_R'

# the file is currently named 'selected_path.csv':
df.to_csv(os.path.join(path, r'xy_all.csv'), index=False)

print('Selected paths saved to csv !')

               ID                              Time        x        y
134959  150012936  2017-05-12 07:54:04.576000+02:00 -5.02914  10.9192
134960  150012936  2017-05-12 07:54:04.829000+02:00   -4.278  10.4455
134961  150012936  2017-05-12 07:54:05.081000+02:00 -3.78509  9.29752
134962  150012936  2017-05-12 07:54:05.334000+02:00 -3.51797  7.91323
134963  150012936  2017-05-12 07:54:05.586000+02:00 -3.17595  6.70549
Selected paths saved to csv !


**Conversion to lat-long coord:**

In [10]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import os
import csv

""" Converts data x-y coordinates into latitudinal and longitudinal positions.
 Specify below the path where the csv file is to be saved. Requires Pandas and Numpy package."""

########################################################################################################################
# lat <=> y, long <=> x
ref_position = np.array([59.916559, 10.734520])
lat = 59.916687
lon = 10.734836
position = [lat, lon]

diff_lat = position[0] - ref_position[0]
diff_lon = position[1] - ref_position[1]

diff_x = 23.41
diff_y = -20

const_lat = np.round(diff_lat/diff_y, 6)
const_lon = np.round(diff_lon/diff_x, 6)

theta = np.pi + np.pi/2

########################################################################################################################

cols = ['ID','Time','x', 'y']

# Add the csv file to be converted here:
df = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_17_R\xy_all.csv', index_col=False, header=None, sep=',', names= cols)
df = df.drop(df.index[0])   # drops the first row

df['lat'] = df['y'].astype(float)
df['long'] = df['x'].astype(float)
df['x'] = df['long'].astype(float)
df['y'] = df['lat'].astype(float)

# rotate x-y-coordinates by pi/2:
#df['x_new'] = df['x'].multiply(np.cos(theta)) - df['y'].multiply(np.sin(theta))
#df['y_new'] = df['x'].multiply(np.sin(theta)) + df['y'].multiply(np.cos(theta))

df['const_lat'] = const_lat
df['const_long'] = const_lon
df['ref_lat'] = ref_position[0]
df['ref_long'] = ref_position[1]

df['lat'] = df['y'].multiply(df['const_lat'])
df['lat'] = df['lat'].add(df['ref_lat'])

df['long'] = df['x'].multiply(df['const_long'])
df['long'] = df['long'].add(df['ref_long'])

#df = df.drop(['const_lat', 'const_long', 'ref_lat', 'ref_long', 'x', 'y', 'x_new', 'y_new'], axis=1)
df = df.drop(['const_lat', 'const_long', 'ref_lat', 'ref_long', 'x', 'y'], axis=1)

# rounding positions coordinates to 6 decimals:
df = df.round({'lat': 6, 'long': 6})

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_17_R'

# the file is currently named 'LatLongData.csv':
df.to_csv(os.path.join(path, r'LatLongData_area_2017_05_17_all.csv'), index=False)

print('conversion completed!')
df_converted = df


conversion completed!


**Creating geojson file:**

In [11]:
import pandas as pd
import numpy as np
import json
import csv


''' This script converts the a data frame with columns: 'ID', 'lat', long' into geojson file for plotting '''

########################################################################################################################
'''
# example data:    note: the 'lat' and 'long' must not be of typ numpy.int64 !
Data = {'ID': [1, 1, 1, 1, 2, 2, 2, 2, 2, 2], 'lat':  [10.734221, 10.734474, 10.734661, 10.734940,  10.734114,
                                                       10.734527, 10.734436, 10.734168, 10.733894, 10.733749],
        'long': [59.916174, 59.916397, 59.916545, 59.916757,  59.916206, 59.916521, 59.916615, 59.916679, 59.916765,
                 59.916798]
        }
df = DataFrame(Data, columns=['ID', 'lat', 'long'])

'''
########################################################################################################################
cols = ['ID','Time','lat', 'long']
# Add the csv file to be converted here:
df = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_17_R\LatLongData_area_2017_05_17_all.csv', index_col=False, header=None, sep=',', names=cols)
df = df.drop(df.index[0])   # drops the first row
del df['Time']
#df = df.loc[:, ['ID', 'lat', 'long']]


# choose number of traffic objects to plot:
num_TO = 900
TO_unique = df.ID.unique()
batch = TO_unique[0:num_TO]
df = df[df['ID'].isin(batch)]

df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)
def df_to_geojson(df):
    # extracting unique TO IDs:
    TO_unique = df.ID.unique()
    MultiLineString = []
    for object in TO_unique:
        # converts the the data frame to an array  [[ , ],[ , ],...[ , ]] :
        string = df.loc[df['ID'] == object].values
        # deleting first element in the array (corresponding to the ID value):
        string = np.delete(string, np.s_[0], 1)
        num_rows = len(string)
        object_string = []
        for row in range(num_rows):
            string[row][0] = np.round(string[row][0], 6)
            string[row][1] = np.round(string[row][1], 6)
            object_string.append([string[row][1], string[row][0]])
        MultiLineString.append(object_string)

    geojson_dict = {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry':
        {'type':'MultiLineString', 'coordinates': MultiLineString}}]}

    return geojson_dict

geojson_dict = df_to_geojson(df)

geojson_str = json.dumps(geojson_dict, indent=2)

# save the geojson result to a file
output_filename = 'TO_paths_2017_05_17_all.geojson'


#with open(output_filename, 'wb') as output_file:
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))

print('geojson created!')

geojson created!
